In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 7 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Challenge : une méthode à pas de temps adaptative
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Challenge : une méthode à pas de temps adaptative

In [ ]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Estimateur d'erreur

Dans un premier temps, nous montrons comment construire un estimateur de l'erreur locale de troncature à partir des itérés de la méthode d'Euler explicite. L'estimateur permet de prédire un pas de temps adaptatif et de réduire le coût de la méthode à précision fixée. Cet estimateur est testé sur l'équation de Hisrchfelder et Curtis; il fonctionne bien quand la tolérance est suffisamment petite mais pose des problèmes quand la tolérance est grande. Le but du challenge est alors d'expliquer ce comportement à l'aide des éléments présentés en amphi.

On rappelle que dans le cours, l'erreur locale de troncature peut s'estimer à partir de la dérivée seconde de la solution $ \mathcal{E}_n = \frac{\Delta t_n^2}{2} y''(t_n) + O(\Delta t_n^3)$. Lorsque l'on utilise un solveur de type Euler-Cauchy ou Euler explicite, montrer qu'un estimateur de l'erreur locale de troncature peut être donné par 

$$
\frac{\Delta t_n}{2}  \left ( f(t_{n}, y_{n}) - f(t_{n-1}, y_{n-1})\right ), \qquad y''(t_n) \simeq \frac{f(t_{n}, y_{n}) - f(t_{n-1}, y_{n-1})}{\Delta t_n}.
$$

Lorsque l'utilisateur veut adapter le pas de temps de manière à obtenir une erreur locale de troncature majorée par une tolérance $\epsilon$ qu'il choisit, montrer que le pas de temps à l'instant $n$ peut être choisi par la formule : 

$$
\Delta t_{n} =   \sqrt{\frac{2\epsilon \Delta t_{n}}{f(t_{n}, y_n) - f(t_{n-1}, y_{n-1})} },
$$

dans la mesure où l'utilisateur fournit aussi un pas de temps initial. 

## Mise en oeuvre sur le problème de Curtiss et Hirschfelder

On considère le problème suivant :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(cos(t) - u(t)) \big) \quad \text{with } k > 1\\ 
u(0)  & = u_0 
\end{aligned} 
\right. 
$$

In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, u) :
        k = self.k
        u_dot = k * (np.cos(t) - u)
        return u_dot

    def sol(self, uini, t0, t):
        k = self.k

        c0 = (uini - (k/(k*k + 1)) * (k*np.cos(t0) + np.sin(t0))) * np.exp(k*t0)
        u = (k/(k*k + 1)) * (k*np.cos(t) + np.sin(t)) +  c0 * np.exp(-k*t)
        return u

### Euler explicite

Dans la suite le lecteur pourra observer cette méthode pour plusieurs tolérances. On pourra remarquer que prendre une tolérance à 10−5 conduit à une solution de qualité, d'un niveau d'erreur inférieur à une solution avec 200 points mais avec un nombre de point environ deux fois plus petit. Choisir une tolérance plus grande 10−4 conduit à des comportements entachés d'erreurs importantes hors de la couche limite en temps initiale. Expliquer ce phénomène.

In [ ]:
#####################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

#####################################################
def forward_euler_adapted(tini, tend, dtini, yini, fcn, tol=1e-6):

    t = [tini];

    yini = np.atleast_1d(yini)
    y = [yini]
        
    dt = dtini
    tn = tini
    it = 0
    
    while (tn<tend):
        yn = y[it]
        y.append(yn + dt*np.array(fcn(tn, yn)))
        tn += dt
        t.append(tn)
        ddy = abs(y[it+1]-yn)/dt
        dt = np.sqrt(2*tol/np.linalg.norm(ddy))
        if (tn+dt)>tend: dt = tend-tn
        it += 1
        
    y = np.stack(y, axis=1)
    t = np.array(t)
    
    return ode_result(y, t)

In [ ]:
uini = (2.,)
tini = 0.
tend = 0.5
k = 100.

cm = curtiss_model(k)
fcn = cm.fcn

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"))

nt = 200
sol = forward_euler(tini, tend, nt, uini, fcn)
err = np.abs(sol.y[0] - cm.sol(uini, tini, sol.t))
print("Euler explicite :", end="")
print(f"  nb de points temporels = {nt}")

color = fig.layout.template.layout.colorway
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='euler', mode='markers+lines', line_color=color[0], legendgroup='sol'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=err, name='euler', mode='markers+lines', line_color=color[0], legendgroup='err'), row=2, col=1)

dtini = 1e-4
tol = [1e-4, 1e-5, 1e-6]
for tol_i in tol:
    sol_adapt = forward_euler_adapted(tini, tend, dtini, uini, fcn, tol=tol_i)
    err_adapt = np.abs(sol_adapt.y[0] - cm.sol(uini, tini, sol_adapt.t))
    print(f"Euler explicite adaptée avec une tolérance à {tol_i:.2e}:", end="")
    print(f"  nb de points temporels = {sol_adapt.t.size}")
    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=sol_adapt.y[0], name='euler<br>adaptée', 
                             line_color=color[1], mode='markers+lines', legendgroup='sol'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=err_adapt, name='euler<br>adaptée', 
                             line_color=color[1], mode='markers+lines', legendgroup='err'), row=2, col=1)

fig.data[2].visible = True
fig.data[3].visible = True

steps = []
for i, tol_i in enumerate(tol):
    args = [{"visible": [(el==0) or (el==1) or (el==2*i+2) or (el==2*i+3) for el in range(len(fig.data))]}]
    step = dict(method="update", label=f"{tol_i:.2e}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'tol = '}, steps=steps)]

legend = dict(x=0.9, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick="toggleitem")
fig.update_xaxes(title='t')
fig.update_layout(height=750, sliders=sliders, legend=legend)